# Trabajo Practico 3: Machine Learning

integrantes: Alvarez Vico Jazmín

             Cortés Conde Titó Javier María
             
             Frassia Fernando 

In [21]:
import scipy as sp
import pandas as pd
import seaborn as sn
import numpy as np
import matplotlib.pyplot as plt
import pylab
from scipy.signal import welch
import scipy.io
import os
from math import log
log2= lambda x:log(x,2)
from collections import defaultdict

def dar_bandas(lista_sujetos):
    elect = lista_sujetos[0][:,1,:]
    welch = sp.signal.welch(elect, noverlap=100,fs= 250, nperseg = 201)
    index = (np.abs(welch[0]-50)).argmin()
    
    df_alphas = pd.DataFrame()
    df_betas = pd.DataFrame()
    df_thetas = pd.DataFrame()
    df_deltas = pd.DataFrame()
    df_gamas = pd.DataFrame()

    i=1
    for sujeto in lista_sujetos:

        potencias_media = []
        for count in range(10):
            welch = sp.signal.welch(sujeto[:,count,:], noverlap=100, fs=250, nperseg = 201)
            frecuencias = welch[0][:index]
            potencias = welch[1][:, :index]
            promedio_potencias = potencias.mean(axis=0)
            potencias_media.append(promedio_potencias)

        potencias_media = np.array(potencias_media)
        medias = potencias_media.mean(axis=0)

        index_delta = (np.abs(frecuencias-4)).argmin()
        delta = medias[:index_delta]
        df_deltas['sujeto'+str(i)] = delta

        index_theta = (np.abs(frecuencias-8)).argmin()
        theta= medias[index_delta+1:index_theta]
        df_thetas['sujeto'+str(i)] = theta

        index_alpha = (np.abs(frecuencias-13)).argmin()
        alpha = medias[index_theta+1:index_alpha]
        df_alphas['sujeto'+str(i)] = alpha

        index_beta = (np.abs(frecuencias-30)).argmin()
        beta = medias[index_alpha+1:index_beta]
        df_betas['sujeto'+str(i)] = beta

        gama = medias[index_beta+1:]
        df_gamas['sujeto'+str(i)] = gama
        i+=1
    
    return (df_alphas, df_betas, df_deltas, df_gamas, df_thetas)

def dar_potXbandas(df_alphas, df_betas, df_deltas, df_gamas, df_thetas):
    potenciasXbanda = pd.DataFrame()
    
    sum_alpha = df_alphas.sum()
    potenciasXbanda['alpha'] = sum_alpha
    sum_beta = df_betas.sum()
    potenciasXbanda['beta'] = sum_beta
    sum_delta = df_deltas.sum()
    potenciasXbanda['delta'] = sum_delta
    sum_gama = df_gamas.sum()
    potenciasXbanda['gama'] = sum_gama
    sum_theta = df_thetas.sum()
    potenciasXbanda['theta'] = sum_theta
    
    factor_normalizacion = sum_alpha+sum_beta+sum_gama+sum_theta + sum_delta 
    
    potenciasXbanda['alpha_norm'] = sum_alpha/factor_normalizacion
    potenciasXbanda['beta_norm'] = sum_beta/factor_normalizacion
    potenciasXbanda['delta_norm'] = sum_delta/factor_normalizacion
    potenciasXbanda['gama_norm'] = sum_gama/factor_normalizacion
    potenciasXbanda['theta_norm'] = sum_theta/factor_normalizacion
    
    return potenciasXbanda

In [3]:
lista_sujetosP = []
matrices = os.listdir('../tp2/matricesP/')
for mat in matrices:
    m = sp.io.loadmat('../tp2/matricesP/' + mat)
    mm = m['data']
    lista_sujetosP.append(mm)
    
       

lista_sujetosS = []
matrices = os.listdir('../tp2/matricesS/')
for mat in matrices:
    m = sp.io.loadmat('../tp2/matricesS/' + mat)
    mm = m['data']
    lista_sujetosS.append(mm)
    
    
    

In [24]:
df_alphas, df_betas, df_deltas, df_gamas, df_thetas = dar_bandas(lista_sujetosP)

matriz_informacion = dar_potXbandas(df_alphas, df_betas, df_deltas, df_gamas, df_thetas)
print(matriz_informacion)


    

                alpha          beta         delta          gama         theta  \
sujeto1  1.650611e-12  2.064808e-12  4.609779e-11  9.292200e-13  8.798807e-12   
sujeto2  1.637910e-12  9.091313e-13  9.900297e-11  2.338660e-13  5.894919e-12   
sujeto3  1.847865e-12  1.012314e-12  4.861052e-11  3.606270e-13  6.426362e-12   
sujeto4  1.642502e-12  2.340214e-12  6.282008e-11  1.225317e-12  4.621129e-12   

         alpha_norm  beta_norm  delta_norm  gama_norm  theta_norm  
sujeto1    0.027722   0.034679    0.774216   0.015606    0.147777  
sujeto2    0.015211   0.008443    0.919429   0.002172    0.054745  
sujeto3    0.031719   0.017376    0.834405   0.006190    0.110309  
sujeto4    0.022609   0.032213    0.864704   0.016866    0.063609  
